In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser

In [171]:
#function to rename columns of tables where rushing is first
def rushcolumnsrename(df,prefix):
    df.rename(columns={'G':f'{prefix}games',
                       'Att':f'{prefix}RushAtt',
                        'Yds':f'{prefix}RushYds',
                        'Avg':f'{prefix}YPA',
                        'TD':f'{prefix}RushTds',
                        'Rec':f'{prefix}Rec',
                        'Yds.1':f'{prefix}RecYds',
                        'Avg.1':f'{prefix}YPC',
                        'TD.1':f'{prefix}RecTds',
                        'Plays':f'{prefix}Plays',
                        'Yds.2':f'{prefix}TotalYds',
                        'Avg.2':f'{prefix}YPT',
                        'TD.2':f'{prefix}TotalTds'}, inplace = True)
    return df

In [63]:
#function to rename columns of tables where receiving is first
def reccolumnsrename(df,prefix):
    df.rename(columns={'G':f'{prefix}games',
                        'Rec':f'{prefix}Rec',
                        'Yds':f'{prefix}RecYds',
                        'Avg':f'{prefix}YPC',
                        'TD':f'{prefix}RecTds',
                        'Att':f'{prefix}RushAtt',
                        'Yds.1':f'{prefix}RushYds',
                        'Avg.1':f'{prefix}YPA',
                        'TD.1':f'{prefix}RushTds',
                        'Plays':f'{prefix}Plays',
                        'Yds.2':f'{prefix}TotalYds',
                        'Avg.2':f'{prefix}YPT',
                        'TD.2':f'{prefix}TotalTds'}, inplace = True)
    return df

In [113]:
#function to append 0's if someone has less than 3 years of stats- for 2 guys.  Don't want to eliminate them yet
def append0less3(df):
    df['dr_2games']=0,
    df['dr_2Rec']=0,
    df['dr_2RecYds']=0,
    df['dr_2YPC']=0,
    df['dr_2RecTds']=0,
    df['dr_2RushAtt']=0,
    df['dr_2RushYds']=0,
    df['dr_2YPA']=0,
    df['dr_2RushTds']=0,
    df['dr_2Plays']=0,
    df['dr_2TotalYds']=0,
    df['dr_2YPT']=0,
    df['dr_2TotalTds']=0,
    df['dr_3games']=0,
    df['dr_3Rec']=0,
    df['dr_3RecYds']=0,
    df['dr_3YPC']=0,
    df['dr_3RecTds']=0,
    df['dr_3RushAtt']=0,
    df['dr_3RushYds']=0,
    df['dr_3YPA']=0,
    df['dr_3RushTds']=0,
    df['dr_3Plays']=0,
    df['dr_3TotalYds']=0,
    df['dr_3YPT']=0,
    df['dr_3TotalTds']=0
    return df

In [116]:
#function to append 0's if someone has less than 4 years of stats. Note: 2 years and 1 career row
def append0less4(df):
    df['dr_3games']=0,
    df['dr_3Rec']=0,
    df['dr_3RecYds']=0,
    df['dr_3YPC']=0,
    df['dr_3RecTds']=0,
    df['dr_3RushAtt']=0,
    df['dr_3RushYds']=0,
    df['dr_3YPA']=0,
    df['dr_3RushTds']=0,
    df['dr_3Plays']=0,
    df['dr_3TotalYds']=0,
    df['dr_3YPT']=0,
    df['dr_3TotalTds']=0
    return df

In [168]:
# Function to parse out individuals college stats and store in a single row with appropriate labels for career,
# draft_1 (year before drafted (Note: draft year is 1 year after season), draft_2 (2 years before drafted),
# draft_3 (years before drafted)

#sample containing a person with 3, 4 and 5 rows for testing
sample = ['Derrick Williams', 'Cordarrelle Patterson','Jeremy Maclin']

def stats1row(indivs,df,colRename):
    #result is returned as a list created here
    rows = []
    #loop through the indexes of a list of individuals
    for i in range(len(indivs)):
        years=[]
        
        #store the individual
        player = indivs[i]
        
        #limit dataframe to results with that individual
        playerdf = df[df['Player']==indivs[i]]
#         print(len(playerdf))
        #store the total rows with stats
        years = playerdf['Year'].unique()
        #Get career data in a dataframe
        career = playerdf[playerdf['Year']==years[-1]]
        #rename columns
        car_row = colRename(career,'career')
        #drop columns
        car_row.drop(columns = ['Year', 'School', 'Conf', 'Class', 'Pos'], inplace = True)
        #get dataframe of 1 year prior to draft
        draft_1 = playerdf[playerdf['Year']==years[-2]]
        #rename columns
        lastyr_row = colRename(draft_1,'dr_1')

        #new dataframe of career and 1 year prior merged
        career_one_row = car_row.merge(lastyr_row, on='Player',how = 'inner')
        
        #check if df has less than 3 rows
        if len(playerdf)<3:
            #if so append zeroes for those columns that can't be appended
            career_one_row = append0less3(career_one_row)
        else:
            #get dataframe of 2 year prior to draft
            draft_2 = playerdf[playerdf['Year']==years[-3]]
            last2yr_row = colRename(draft_2,'dr_2')
            last2yr_row.drop(columns = ['Year', 'School', 'Conf', 'Class', 'Pos'], inplace = True)

            #merge 2 years prior
            career_one_row= career_one_row.merge(last2yr_row, on = 'Player', how = 'inner')
            
            #check if df has less than 4 rows
            if len(playerdf)<4:
                #if so append zeroes for those columns that can't be appended
                career_one_row = append0less4(career_one_row)
            else:
                #get dataframe of 3 years prior to draft
                draft_3 = playerdf[playerdf['Year']==years[-4]]
                last3yr_row = colRename(draft_3,'dr_3')
                last3yr_row.drop(columns = ['Year', 'School', 'Conf', 'Class', 'Pos'], inplace = True)

                #merge 3 years prior
                career_one_row= career_one_row.merge(last3yr_row, on = 'Player', how = 'inner')
#             print('--------------------')

        rows.append(career_one_row)
    return rows
#     print(lastyr_row)

In [2]:
#Overall the purpose of this is to scrape information on players and store names and urls of players that were drafted
# and they have college stats on record.  Note: this made me limit my exploration to D1 players because D2 player statistics
# were much more individualized.  In the future I would like to expand my dataset to include those individuals so I have
# stored the record of those individuals that were drafted, but didn't have college stats

#spin up web browser to visit different combine results pages by year
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
#Create a list of years of combine stats that I will gather
years = ['2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020']
#list to store raw dataframes in case I need to reference later
dfs=[]
#place to store dictionaries of players and the url's to their statistics
links = []
#list to store the information of players that don't have links to stats
p_no_stats = []
for yr in years:
    print(yr)
    print('-------------------------------')
    #insert years into url, visit url, and turn into a soup object
    url = f"https://www.pro-football-reference.com/play-index/nfl-combine-results.cgi?request=1&year_min={yr}&year_max={yr}&height_min=60&height_max=82&weight_min=120&weight_max=400&pos%5B%5D=WR&show=all&order_by=year_id"
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    #separate out all html elements with label data-stat = college which corresponds to the number of indivs on the page
    a= soup.find_all(attrs={"data-stat": "college"})
    
    #loop through by index to check all rows of data stored in the combine table
    for i in range(len(a)):
        
        #for each grab and store the element that has the player name
        name = soup.find_all(attrs ={"data-stat":"player"})
        #store the value that indicates if a player was drafted, the round, pick, and team
        draft = soup.find_all(attrs = {'data-stat':'draft_info'})
        
        #assign the player name to variable player
        player = name[i].text
        #assign the drafted text to variable drafted
        drafted = draft[i].text
        
        #check if length of drafted is 0, which indicates that there is no record or the player wasn't drafted
        if len(drafted) == 0:
            #print out the player name to check functioning. loop returns
            print(f'{player} was not drafted')
        
        #if a player was drafted do the following
        else:
            #try catch to attempt to get the link to college stats
            try: 
                #get url of player
                link = a[i].find('a')['href']
                #print and store a combination of player and url
                print(f'{player} drafted college stats here: {link}')
                links.append({'player':player,'url':link})
            #handling players without a url    
            except:
                #separate out header rows so they are not stored
                if player == 'Player':
                    print('header row')
                #store the info of players that were drafted, but didn't have college stats
                else:
                    print(f"{player} No college Stats")
                    p_no_stats.append(player)
    print("Next Year")




2009
-------------------------------
header row
Derrick Williams drafted college stats here: https://www.sports-reference.com/cfb/players/derrick-williams-2.html
Mike Wallace drafted college stats here: https://www.sports-reference.com/cfb/players/mike-wallace-1.html
Tiquan Underwood drafted college stats here: https://www.sports-reference.com/cfb/players/tiquan-underwood-1.html
Patrick Turner drafted college stats here: https://www.sports-reference.com/cfb/players/patrick-turner-1.html
Mike Thomas drafted college stats here: https://www.sports-reference.com/cfb/players/mike-thomas-9.html
Brandon Tate drafted college stats here: https://www.sports-reference.com/cfb/players/brandon-tate-1.html
Sammie Stroughter drafted college stats here: https://www.sports-reference.com/cfb/players/sammie-stroughter-1.html
Demetrius Byrd drafted college stats here: https://www.sports-reference.com/cfb/players/demetrius-byrd-1.html
Brian Robiskie drafted college stats here: https://www.sports-reference.

David Gettis drafted college stats here: https://www.sports-reference.com/cfb/players/david-gettis-1.html
Jacoby Ford drafted college stats here: https://www.sports-reference.com/cfb/players/jacoby-ford-1.html
Armanti Edwards No college Stats
Marcus Easley drafted college stats here: https://www.sports-reference.com/cfb/players/marcus-easley-1.html
Eric Decker drafted college stats here: https://www.sports-reference.com/cfb/players/eric-decker-1.html
Riley Cooper drafted college stats here: https://www.sports-reference.com/cfb/players/riley-cooper-1.html
Dez Bryant drafted college stats here: https://www.sports-reference.com/cfb/players/dez-bryant-1.html
header row
Antonio Brown drafted college stats here: https://www.sports-reference.com/cfb/players/antonio-brown-1.html
Dezmon Briscoe drafted college stats here: https://www.sports-reference.com/cfb/players/dezmon-briscoe-1.html
Arrelious Benn drafted college stats here: https://www.sports-reference.com/cfb/players/arrelious-benn-1.htm

Julio Jones drafted college stats here: https://www.sports-reference.com/cfb/players/julio-jones-1.html
Ronald Johnson drafted college stats here: https://www.sports-reference.com/cfb/players/ronald-johnson-2.html
Jerrel Jernigan drafted college stats here: https://www.sports-reference.com/cfb/players/jerrel-jernigan-1.html
Lestar Jean was not drafted
Andre Holmes was not drafted
Dwayne Harris drafted college stats here: https://www.sports-reference.com/cfb/players/dwayne-harris-1.html
Leonard Hankerson drafted college stats here: https://www.sports-reference.com/cfb/players/leonard-hankerson-1.html
Jamel Hamler was not drafted
Tori Gurley was not drafted
header row
A.J. Green drafted college stats here: https://www.sports-reference.com/cfb/players/aj-green-1.html
Clyde Gates No college Stats
Tandon Doss drafted college stats here: https://www.sports-reference.com/cfb/players/tandon-doss-1.html
Mark Dell was not drafted
Randall Cobb drafted college stats here: https://www.sports-refere

header row
Devon Wylie drafted college stats here: https://www.sports-reference.com/cfb/players/devon-wylie-1.html
Kendall Wright drafted college stats here: https://www.sports-reference.com/cfb/players/kendall-wright-1.html
Jarius Wright drafted college stats here: https://www.sports-reference.com/cfb/players/jarius-wright-1.html
Jordan White drafted college stats here: https://www.sports-reference.com/cfb/players/jordan-white-1.html
Nick Toon drafted college stats here: https://www.sports-reference.com/cfb/players/nick-toon-1.html
Tommy Streeter drafted college stats here: https://www.sports-reference.com/cfb/players/tommy-streeter-1.html
Mohamed Sanu drafted college stats here: https://www.sports-reference.com/cfb/players/mohamed-sanu-1.html
James Rodgers was not drafted
Rueben Randle drafted college stats here: https://www.sports-reference.com/cfb/players/rueben-randle-1.html
Brian Quick No college Stats
DeVier Posey drafted college stats here: https://www.sports-reference.com/cfb/

header row
Robert Woods drafted college stats here: https://www.sports-reference.com/cfb/players/robert-woods-1.html
Marquess Wilson drafted college stats here: https://www.sports-reference.com/cfb/players/marquess-wilson-1.html
Terrance Williams drafted college stats here: https://www.sports-reference.com/cfb/players/terrance-williams-2.html
Markus Wheaton drafted college stats here: https://www.sports-reference.com/cfb/players/markus-wheaton-1.html
Conner Vernon was not drafted
Kenbrell Thompkins was not drafted
Ryan Swope drafted college stats here: https://www.sports-reference.com/cfb/players/ryan-swope-1.html
Kenny Stills drafted college stats here: https://www.sports-reference.com/cfb/players/kenny-stills-1.html
Ryan Spadola was not drafted
Rodney Smith was not drafted
Ace Sanders drafted college stats here: https://www.sports-reference.com/cfb/players/ace-sanders-1.html
Lanear Sampson was not drafted
Da'Rick Rogers was not drafted
Denard Robinson drafted college stats here: http

...

header row
De'Runnya Wilson was not drafted
Duke Williams was not drafted
Laquon Treadwell drafted college stats here: https://www.sports-reference.com/cfb/players/laquon-treadwell-1.html
Michael Thomas drafted college stats here: https://www.sports-reference.com/cfb/players/michael-thomas-3.html
Nelson Spruce was not drafted
Sterling Shepard drafted college stats here: https://www.sports-reference.com/cfb/players/sterling-shepard-1.html
Tajae Sharpe drafted college stats here: https://www.sports-reference.com/cfb/players/tajae-sharpe-1.html
Hunter Sharp was not drafted
Rashawn Scott was not drafted
Alonzo Russell was not drafted
Demarcus Robinson drafted college stats here: https://www.sports-reference.com/cfb/players/demarcus-robinson-3.html
Charone Peake drafted college stats here: https://www.sports-reference.com/cfb/players/charone-peake-1.html
Jordan Payton drafted college stats here: https://www.sports-reference.com/cfb/players/jordan-payton-1.html
Chris Moore drafted college st

header row
Jesus Wilson was not drafted
Mike Williams drafted college stats here: https://www.sports-reference.com/cfb/players/mike-williams-17.html
Kermit Whitfield was not drafted
Dede Westbrook drafted college stats here: https://www.sports-reference.com/cfb/players/dede-westbrook-1.html
Greg Ward Jr. was not drafted
Noel Thomas was not drafted
Trent Taylor drafted college stats here: https://www.sports-reference.com/cfb/players/trent-taylor-2.html
Taywan Taylor drafted college stats here: https://www.sports-reference.com/cfb/players/taywan-taylor-1.html
Ryan Switzer drafted college stats here: https://www.sports-reference.com/cfb/players/ryan-switzer-1.html
ArDarius Stewart drafted college stats here: https://www.sports-reference.com/cfb/players/ardarius-stewart-1.html
Jamari Staples was not drafted
JuJu Smith-Schuster drafted college stats here: https://www.sports-reference.com/cfb/players/juju-smith-1.html
Ricky Seals-Jones was not drafted
Artavis Scott was not drafted
Curtis Sam

header row
Javon Wims drafted college stats here: https://www.sports-reference.com/cfb/players/javon-wims-1.html
Cedrick Wilson drafted college stats here: https://www.sports-reference.com/cfb/players/cedric-wilson-3.html
Jester Weah was not drafted
James Washington drafted college stats here: https://www.sports-reference.com/cfb/players/james-washington-5.html
Marquez Valdes-Scantling drafted college stats here: https://www.sports-reference.com/cfb/players/marquez-valdes-scantling-1.html
Auden Tate drafted college stats here: https://www.sports-reference.com/cfb/players/auden-tate-1.html
Courtland Sutton drafted college stats here: https://www.sports-reference.com/cfb/players/courtland-sutton-1.html
Equanimeous St. brown drafted college stats here: https://www.sports-reference.com/cfb/players/equanimeous-st-brown-1.html
Jaleel Scott drafted college stats here: https://www.sports-reference.com/cfb/players/jaleel-scott-1.html
Korey Robertson was not drafted
Calvin Ridley drafted college

header row
Antoine Wesley was not drafted
Jamarius Way was not drafted
Cody Thompson was not drafted
Jaylen Smith was not drafted
Darius Slayton drafted college stats here: https://www.sports-reference.com/cfb/players/darius-slayton-1.html
David Sills was not drafted
Deebo Samuel drafted college stats here: https://www.sports-reference.com/cfb/players/deebo-samuel-1.html
Riley Ridley drafted college stats here: https://www.sports-reference.com/cfb/players/riley-ridley-1.html
Hunter Renfrow drafted college stats here: https://www.sports-reference.com/cfb/players/hunter-renfrow-1.html
Nyqwan Murray was not drafted
Stanley Morgan was not drafted
Dillon Mitchell drafted college stats here: https://www.sports-reference.com/cfb/players/dillon-mitchell-1.html
Jakobi Meyers was not drafted
D.K. Metcalf drafted college stats here: https://www.sports-reference.com/cfb/players/dk-metcalf-1.html
Terry McLaurin drafted college stats here: https://www.sports-reference.com/cfb/players/terry-mclaurin-

header row
Cody White was not drafted
Quez Watkins drafted college stats here: https://www.sports-reference.com/cfb/players/quez-watkins-1.html
Ben Victor was not drafted
Jeff Thomas was not drafted
Freddie Swain drafted college stats here: https://www.sports-reference.com/cfb/players/freddie-swain-1.html
Darrell Stewart was not drafted
Laviska Shenault Jr. drafted college stats here: https://www.sports-reference.com/cfb/players/laviska-shenault-jr-1.html
Henry Ruggs III drafted college stats here: https://www.sports-reference.com/cfb/players/henry-ruggs-iii-1.html
Kendrick Rogers was not drafted
Joe Reed drafted college stats here: https://www.sports-reference.com/cfb/players/joe-reed-4.html
Jalen Reagor drafted college stats here: https://www.sports-reference.com/cfb/players/jalen-reagor-1.html
James Proche drafted college stats here: https://www.sports-reference.com/cfb/players/james-proche-1.html
Michael Pittman drafted college stats here: https://www.sports-reference.com/cfb/playe

In [4]:
#variable storing a few players name and url's for creating, troubleshooting, and maybe adjustments later on
couplelinks = [{'player': 'Lynn Bowden', 'url': 'https://www.sports-reference.com/cfb/players/lynn-bowden-jr-1.html'},
{'player': 'Brandon Aiyuk', 'url': 'https://www.sports-reference.com/cfb/players/brandon-aiyuk-1.html'},
               {'player': 'Chase Claypool', 'url': 'https://www.sports-reference.com/cfb/players/chase-claypool-1.html'}]

In [8]:
#larger subset of 2009 data for larger testing, troubleshooting
links2009 = [{'player': 'Derrick Williams', 'url': 'https://www.sports-reference.com/cfb/players/derrick-williams-2.html'}, {'player': 'Mike Wallace', 'url': 'https://www.sports-reference.com/cfb/players/mike-wallace-1.html'}, {'player': 'Tiquan Underwood', 'url': 'https://www.sports-reference.com/cfb/players/tiquan-underwood-1.html'}, {'player': 'Patrick Turner', 'url': 'https://www.sports-reference.com/cfb/players/patrick-turner-1.html'}, {'player': 'Mike Thomas', 'url': 'https://www.sports-reference.com/cfb/players/mike-thomas-9.html'}, {'player': 'Brandon Tate', 'url': 'https://www.sports-reference.com/cfb/players/brandon-tate-1.html'}, {'player': 'Sammie Stroughter', 'url': 'https://www.sports-reference.com/cfb/players/sammie-stroughter-1.html'}, {'player': 'Demetrius Byrd', 'url': 'https://www.sports-reference.com/cfb/players/demetrius-byrd-1.html'}, {'player': 'Brian Robiskie', 'url': 'https://www.sports-reference.com/cfb/players/brian-robiskie-1.html'}, {'player': 'Hakeem Nicks', 'url': 'https://www.sports-reference.com/cfb/players/hakeem-nicks-1.html'}, {'player': 'Louis Murphy', 'url': 'https://www.sports-reference.com/cfb/players/louis-murphy-1.html'}, {'player': 'Marko Mitchell', 'url': 'https://www.sports-reference.com/cfb/players/marko-mitchell-1.html'}, {'player': 'Kenny McKinley', 'url': 'https://www.sports-reference.com/cfb/players/kenny-mckinley-1.html'}, {'player': 'Mohamed Massaquoi', 'url': 'https://www.sports-reference.com/cfb/players/mohamed-massaquoi-1.html'}, {'player': 'Jeremy Maclin', 'url': 'https://www.sports-reference.com/cfb/players/jeremy-maclin-1.html'}, {'player': 'Manuel Johnson', 'url': 'https://www.sports-reference.com/cfb/players/manuel-johnson-1.html'}, {'player': 'Juaquin Iglesias', 'url': 'https://www.sports-reference.com/cfb/players/juaquin-iglesias-1.html'}, {'player': 'Darrius Heyward-Bey', 'url': 'https://www.sports-reference.com/cfb/players/darrius-heyward-bey-1.html'}, {'player': 'Percy Harvin', 'url': 'https://www.sports-reference.com/cfb/players/percy-harvin-1.html'}, {'player': 'Brian Hartline', 'url': 'https://www.sports-reference.com/cfb/players/brian-hartline-1.html'}, {'player': 'Brandon Gibson', 'url': 'https://www.sports-reference.com/cfb/players/brandon-gibson-1.html'}, {'player': 'Brooks Foster', 'url': 'https://www.sports-reference.com/cfb/players/brooks-foster-1.html'}, {'player': 'Jarett Dillard', 'url': 'https://www.sports-reference.com/cfb/players/jarett-dillard-1.html'}, {'player': 'Michael Crabtree', 'url': 'https://www.sports-reference.com/cfb/players/michael-crabtree-1.html'}, {'player': 'Austin Collie', 'url': 'https://www.sports-reference.com/cfb/players/austin-collie-1.html'}, {'player': 'Deon Butler', 'url': 'https://www.sports-reference.com/cfb/players/deon-butler-1.html'}, {'player': 'Kenny Britt', 'url': 'https://www.sports-reference.com/cfb/players/kenny-britt-1.html'}]

In [15]:
# The following code spins up a browser and visits the urls of individual players to collect their college stats

#Open a browser window
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

#Lists to store the dfs, the people without a table of rec stats, and info about player and number of stat rows
dfs = []
p_no_recs=[]
records = []

#Loop that is going through a dictionary of players names a the links that were previously scraped and stored 
#for players that were drafted and have college stats available
for i in range(len(links)):
#     print(links[i]['url'])

    #try catch that will store all players for which I can't identify a receiving stats table
    try:
        #visit url of player
        browser.visit(links[i]['url'])
#         print('visited')
        #grab html and turn into a soup object
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
#         print('soup object create')
        #locate the receiving stats table
        table = soup.find(id ='all_receiving')
#         print('table found')
        #convert html of receiving stats table to a pandas dataframe
        df = pd.read_html(str(table), header = 1)
#         print('dataframe read')
        #store the player variable from the links dictionary
        player = links[i]['player']
        #append the players name to the dataframe
        df[0]['Player']=player
#         print('name column created')
        #create a new dataframe
        playerdf=df[0]
#         print(len(playerdf))
        #strip the * from players stats that denote they played in a bowl game
        playerdf['Year']=playerdf['Year'].map(lambda x: x.lstrip('*'))
        
        #get and store the number of rows of stats in the dataframe for later use
        years = playerdf['Year'].unique()
        datarow = len(years)
        #append a record for each player along with rows of stats
        records.append({'Player':links[i]['player'],'Stat Rows':len(years)})
        #print for tracking and checking that scraping is going as expected
        print(f'{player}:{len(years)} years stats')
        
        dfs.append(playerdf)

    except:
        #if no receiving table is found or something goes wrong store the record and url for further investigation
        player = links[i]['player']
        url =links[i]['url']
        print(f'{player}:No receiving stats')
        p_no_recs.append({'player': player,'url':url})

https://www.sports-reference.com/cfb/players/derrick-williams-2.html
Derrick Williams:5 years stats
https://www.sports-reference.com/cfb/players/mike-wallace-1.html
Mike Wallace:4 years stats
https://www.sports-reference.com/cfb/players/tiquan-underwood-1.html
Tiquan Underwood:5 years stats
https://www.sports-reference.com/cfb/players/patrick-turner-1.html
Patrick Turner:5 years stats
https://www.sports-reference.com/cfb/players/mike-thomas-9.html
Mike Thomas:5 years stats
https://www.sports-reference.com/cfb/players/brandon-tate-1.html
Brandon Tate:5 years stats
https://www.sports-reference.com/cfb/players/sammie-stroughter-1.html
Sammie Stroughter:6 years stats
https://www.sports-reference.com/cfb/players/demetrius-byrd-1.html
Demetrius Byrd:3 years stats
https://www.sports-reference.com/cfb/players/brian-robiskie-1.html
Brian Robiskie:5 years stats
https://www.sports-reference.com/cfb/players/hakeem-nicks-1.html
Hakeem Nicks:4 years stats
https://www.sports-reference.com/cfb/players

A.J. Jenkins:5 years stats
https://www.sports-reference.com/cfb/players/alshon-jeffery-1.html
Alshon Jeffery:4 years stats
https://www.sports-reference.com/cfb/players/ty-hilton-1.html
T.Y. Hilton:5 years stats
https://www.sports-reference.com/cfb/players/stephen-hill-1.html
Stephen Hill:4 years stats
https://www.sports-reference.com/cfb/players/junior-hemingway-1.html
Junior Hemingway:6 years stats
https://www.sports-reference.com/cfb/players/tj-graham-1.html
T.J. Graham:5 years stats
https://www.sports-reference.com/cfb/players/chris-givens-2.html
Chris Givens:4 years stats
https://www.sports-reference.com/cfb/players/michael-floyd-1.html
Michael Floyd:5 years stats
https://www.sports-reference.com/cfb/players/bj-cunningham-1.html
B.J. Cunningham:5 years stats
https://www.sports-reference.com/cfb/players/juron-criner-1.html
Juron Criner:5 years stats
https://www.sports-reference.com/cfb/players/danny-coale-1.html
Danny Coale:5 years stats
https://www.sports-reference.com/cfb/players/

Amari Cooper:4 years stats
https://www.sports-reference.com/cfb/players/chris-conley-1.html
Chris Conley:5 years stats
https://www.sports-reference.com/cfb/players/sammie-coates-1.html
Sammie Coates:4 years stats
https://www.sports-reference.com/cfb/players/kaelin-clay-1.html
Kaelin Clay:2 years stats
https://www.sports-reference.com/cfb/players/daron-brown-1.html
Da'Ron Brown:5 years stats
https://www.sports-reference.com/cfb/players/kenny-bell-2.html
Kenny Bell:5 years stats
https://www.sports-reference.com/cfb/players/mario-alford-1.html
Mario Alford:3 years stats
https://www.sports-reference.com/cfb/players/nelson-agholor-1.html
Nelson Agholor:4 years stats
https://www.sports-reference.com/cfb/players/laquon-treadwell-1.html
Laquon Treadwell:4 years stats
https://www.sports-reference.com/cfb/players/michael-thomas-3.html
Michael Thomas:4 years stats
https://www.sports-reference.com/cfb/players/sterling-shepard-1.html
Sterling Shepard:5 years stats
https://www.sports-reference.com/c

Dillon Mitchell:4 years stats
https://www.sports-reference.com/cfb/players/dk-metcalf-1.html
D.K. Metcalf:4 years stats
https://www.sports-reference.com/cfb/players/terry-mclaurin-1.html
Terry McLaurin:5 years stats
https://www.sports-reference.com/cfb/players/olabisi-johnson-1.html
Bisi Johnson:5 years stats
https://www.sports-reference.com/cfb/players/keesean-johnson-1.html
Keesean Johnson:5 years stats
https://www.sports-reference.com/cfb/players/diontae-johnson-1.html
Diontae Johnson:4 years stats
https://www.sports-reference.com/cfb/players/gary-jennings-1.html
Gary Jennings:5 years stats
https://www.sports-reference.com/cfb/players/andy-isabella-1.html
Andy Isabella:5 years stats
https://www.sports-reference.com/cfb/players/jalen-hurd-1.html
Jalen Hurd:No receiving stats
https://www.sports-reference.com/cfb/players/nkeal-harry-1.html
N'Keal Harry:4 years stats
https://www.sports-reference.com/cfb/players/kelvin-harmon-1.html
Kelvin Harmon:4 years stats
https://www.sports-referenc

In [134]:
# The following code spins up a browser and visits the urls individual players for which a receiving table wasn't found
# because it appears their table was labeled rushing and their rushing and receiving stats were reversed in table storage

p_no_recs_dfs = []
p_no_recs_or_rush=[]
p_no_recs_records = []

#Open a browser window
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

#loop through the records that were drafted, but didn't return a receiving stats table
for i in range(len(p_no_recs)):
    print(p_no_recs[i])

    #try catch that will store all players for which I can't identify a rushing stats table
    try:
        #visit url of player
        browser.visit(p_no_recs[i]['url'])
#         print('visited')
        #grab html and turn into a soup object
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
#         print('soup object create')
        #locate the receiving stats table
        table = soup.find(id ='all_rushing')
#         print('table found')
        #convert html of receiving stats table to a pandas dataframe
        df = pd.read_html(str(table), header = 1)
#         print('dataframe read')
        #store the player variable from the p_no_recs dictionary
        player = p_no_recs[i]['player']
        #append the players name to the dataframe
        df[0]['Player']=player
#         print('name column created')
        #create a new dataframe
        playerdf=df[0]
#         print(len(playerdf))
        #strip the * from players stats that denote they played in a bowl game
        playerdf['Year']=playerdf['Year'].map(lambda x: x.lstrip('*'))
        
        #get and store the number of rows of stats in the dataframe for later use
        years = playerdf['Year'].unique()
        datarow = len(years)
        #append a record for each player along with rows of stats
        p_no_recs_records.append({'Player':p_no_recs[i]['player'],'Stat Rows':len(years)})
        #print for tracking and checking that scraping is going as expected
        print(f'{player}:{len(years)} years stats')
        
        p_no_recs_dfs.append(playerdf)

    except:
        #if no receiving table is found or something goes wrong store the record and url for further investigation
        player = p_no_recs[i]['player']
        url =p_no_recs[i]['url']
        print(f'{player}:No receiving stats')
        p_no_recs_or_rush.append({'player': player,'url':url})

{'player': 'Percy Harvin', 'url': 'https://www.sports-reference.com/cfb/players/percy-harvin-1.html'}
Percy Harvin:4 years stats
{'player': 'Damian Williams', 'url': 'https://www.sports-reference.com/cfb/players/damian-williams-1.html'}
Damian Williams:No receiving stats
{'player': 'Dexter McCluster', 'url': 'https://www.sports-reference.com/cfb/players/dexter-mccluster-1.html'}
Dexter McCluster:5 years stats
{'player': 'Trindon Holliday', 'url': 'https://www.sports-reference.com/cfb/players/trindon-holliday-1.html'}
Trindon Holliday:5 years stats
{'player': 'Greg Little', 'url': 'https://www.sports-reference.com/cfb/players/greg-little-1.html'}
Greg Little:4 years stats
{'player': 'Randall Cobb', 'url': 'https://www.sports-reference.com/cfb/players/randall-cobb-1.html'}
Randall Cobb:4 years stats
{'player': 'Denard Robinson', 'url': 'https://www.sports-reference.com/cfb/players/denard-robinson-1.html'}
Denard Robinson:5 years stats
{'player': 'Jalen Saunders', 'url': 'https://www.spor

--------------------


C:\Users\walsh\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\walsh\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------


,careergames,careerRushAtt,careerRushYds,careerYPA,careerRushTds,careerRec,careerRecYds,careerYPC,careerRecTds,careerPlays,...,dr_3YPA,dr_3RushTds,dr_3Rec,dr_3RecYds,dr_3YPC,dr_3RecTds,dr_3Plays,dr_3TotalYds,dr_3YPT,dr_3TotalTds
0,NaN,194,1852,9.5,19,133.0,1929.0,14.5,13.0,327,...,10.4,3,34.0,427.0,12.6,2.0,75,855,11.4,5
0,NaN,304,1955,6.4,15,130.0,1703.0,13.1,7.0,434,...,10.5,0,27.0,326.0,12.1,2.0,33,389,11.8,2
0,NaN,115,776,6.7,4,7.0,72.0,10.3,0.0,122,...,6.9,2,2.0,3.0,1.5,0.0,55,367,6.7,2
0,NaN,166,805,4.8,6,86.0,969.0,11.3,6.0,252,...,5.1,2,13.0,99.0,7.6,1.0,72,399,5.5,3
0,NaN,228,1313,5.8,22,144.0,1661.0,11.5,13.0,372,...,4.0,7,21.0,197.0,9.4,2.0,100,513,5.1,9
0,NaN,723,4495,6.2,42,3.0,31.0,10.3,0.0,726,...,6.6,14,NaN,NaN,NaN,NaN,256,1702,6.6,14
0,NaN,325,2342,7.2,24,99.0,1194.0,12.1,12.0,424,...,3.5,1,16.0,75.0,4.7,1.0,56,215,3.8,2
0,NaN,600,3315,5.5,33,25.0,340.0,13.6,3.0,625,...,6.2,12,0.0,0.0,NaN,0.0,171,1068,6.2,12
0,NaN,172,1286,7.5,15,107.0,1249.0,11.7,9.0,279,...,6.6,6,11.0,95.0,8.6,0.0,69,478,6.9,6
0,NaN,614,4359,7.1,40,22.0,470.0,21.4,3.0,636,...,8.6,11,13.0,303.0,23.3,2.0,151,1485,9.8,13


In [207]:
#Players who played multiple years

# The following code spins up a browser and visits the urls of individual players to collect their college stats

#Open a browser window
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

#Lists to store the redo_dfs, the people without a table of rec stats, and info about player and number of stat rows
redo_dfs = []
no_recs=[]


#Loop that is going through a dictionary of players names a the p_no_recs_or_rush that were previously scraped and stored 
#for players that were drafted and have college stats available
for i in range(len(p_no_recs_or_rush)):
#     print(p_no_recs_or_rush[i]['url'])

    #try catch that will store all players for which I can't identify a receiving stats table
    try:
        #visit url of player
        browser.visit(p_no_recs_or_rush[i]['url'])
#         print('visited')
        #grab html and turn into a soup object
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
#         print('soup object create')
        #locate the receiving stats table
        table = soup.find(id ='all_receiving')
#         print('table found')
        #convert html of receiving stats table to a pandas dataframe
        df = pd.read_html(str(table), header = 1)
#         print('dataframe read')
        #store the player variable from the p_no_recs_or_rush dictionary
        player = p_no_recs_or_rush[i]['player']
#         print("player variable stored")
        #append the players name to the dataframe
        df[0]['Player']=player
#         print('name column created')
        #create a new dataframe
        playerdf=df[0]
#         print(len(playerdf))
        playerdf['Year'].fillna('No Year', inplace = True)
#         print("nulls filled for year")
        #strip the * from players stats that denote they played in a bowl game
        playerdf['Year']=playerdf['Year'].map(lambda x: x.lstrip('*'))
#         print("* stripped from year column")
        #get and store the number of rows of stats in the dataframe for later use
        years = playerdf['Year'].unique()
#         print("unique years identified")
        datarow = len(years)

        #print for tracking and checking that scraping is going as expected
#         print(f'{player}:{len(years)} years stats')
        
        redo_dfs.append(playerdf)

    except:
        #if no receiving table is found or something goes wrong store the record and url for further investigation
        player = p_no_recs_or_rush[i]['player']
        url =p_no_recs_or_rush[i]['url']
#         print(f'{player}:No receiving stats')
        no_recs.append({'player': player,'url':url})

In [209]:
redo_dfs

[      Year    School    Conf Class  Pos     G  Rec   Yds   Avg  TD  Att  \
 0     2006  Arkansas     SEC    FR   WR  13.0   19   235  12.4   2    1   
 1     2008       USC  Pac-10   NaN   WR  13.0   58   869  15.0   9    4   
 2     2009       USC  Pac-10   NaN   WR  12.0   70  1010  14.4   6    2   
 3   Career   Overall     NaN   NaN  NaN   NaN  147  2114  14.4  17    7   
 4  No Year  Arkansas     NaN   NaN  NaN   NaN   19   235  12.4   2    1   
 5  No Year       USC     NaN   NaN  NaN   NaN  128  1879  14.7  15    6   
 
    Yds.1  Avg.1  TD.1  Plays  Yds.2  Avg.2  TD.2           Player  
 0      5    5.0     0     20    240   12.0     2  Damian Williams  
 1     43   10.8     0     62    912   14.7     9  Damian Williams  
 2     16    8.0     0     72   1026   14.3     6  Damian Williams  
 3     64    9.1     0    154   2178   14.1    17  Damian Williams  
 4      5    5.0     0     20    240   12.0     2  Damian Williams  
 5     59    9.8     0    134   1938   14.5    15  D

In [181]:
mult_schools = pd.concat(redo_dfs)
mult_schools['Year'].fillna('subtot',inplace=True)
mult_schools_final = mult_schools[mult_schools['Year']!= 'subtot']
mult_school_one_row = 
mult_schools_final.to_csv("mult_schools.csv")

In [225]:
mult_schools_final

,Year,School,Conf,Class,Pos,G,Rec,Yds,Avg,TD,Att,Yds.1,Avg.1,TD.1,Plays,Yds.2,Avg.2,TD.2,Player
0,2006,Arkansas,SEC,FR,WR,13.0,19.0,235.0,12.4,2.0,1.0,5.0,5.0,0.0,20,240,12.0,2,Damian Williams
1,2008,USC,Pac-10,NaN,WR,13.0,58.0,869.0,15.0,9.0,4.0,43.0,10.8,0.0,62,912,14.7,9,Damian Williams
2,2009,USC,Pac-10,NaN,WR,12.0,70.0,1010.0,14.4,6.0,2.0,16.0,8.0,0.0,72,1026,14.3,6,Damian Williams
3,Career,Overall,NaN,NaN,NaN,NaN,147.0,2114.0,14.4,17.0,7.0,64.0,9.1,0.0,154,2178,14.1,17,Damian Williams
4,No Year,Arkansas,NaN,NaN,NaN,NaN,19.0,235.0,12.4,2.0,1.0,5.0,5.0,0.0,20,240,12.0,2,Damian Williams
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,2018,Florida,SEC,JR,WR,11.0,35.0,503.0,14.4,6.0,0.0,0.0,NaN,0.0,35,503,14.4,6,Van Jefferson
4,2019,Florida,SEC,SR,WR,12.0,49.0,657.0,13.4,6.0,1.0,7.0,7.0,0.0,50,664,13.3,6,Van Jefferson
5,Career,Overall,NaN,NaN,NaN,NaN,175.0,2159.0,12.3,16.0,6.0,47.0,7.8,0.0,181,2206,12.2,16,Van Jefferson
6,No Year,Ole Miss,NaN,NaN,NaN,NaN,91.0,999.0,11.0,4.0,5.0,40.0,8.0,0.0,96,1039,10.8,4,Van Jefferson


In [228]:
#Get a list of unique players
check= all_df['Player'].unique()

players_stat_yrs = []
players = []
for i in range(len(check)):
    years=[]
    players.append(check[i])
    playerdf = all_df[all_df['Player']==check[i]]
    yrs_played = len(playerdf)
    players_stat_yrs.append(yrs_played)
    
#create data frame of results to understand number of years of stats of players
yearsofstats_df = pd.DataFrame({'players':players,'statyears':players_stat_yrs})
#Group by statyears to know the number of players with corresponding stat years
#Note the 9 is Mike Williams which is actually 2 players 1 from drafted by the Bucs and the other Stud from Clemson Go TIGERS!
print(yearsofstats_df.groupby('statyears').count())

#look at record of 2 players to see if valid to just exclude since it represents 2 players that only have 1 year of stats
years2 = yearsofstats_df[yearsofstats_df['statyears']==2]
print(years2)

#look at player with 9 rows.  It is actually Mike Williams which pertains to 2 players so creating separate dataframe below
years9 = yearsofstats_df[yearsofstats_df['statyears']==9]
print(years2)

#get all guys that are primarily receivers and played at a single school into a df with 1 row per player stats
rec_guys = stats1row(check,all_df,reccolumnsrename)

#create dataframe of the list of dataframes created with function
rec_final = pd.concat(rec_guys)
#check the length to validate the function worked properly
len(rec_final)
#Exclude Mike Williams
minus_mikew = rec_final[rec_final['Player']!='Mike Williams']
#save as CSV file
minus_mikew.to_csv("recfinal.csv")

# Get a dataframe consisting of only Mike Williams from Clemson Info
clem_mike_w = all_df[(all_df['Player']=='Mike Williams')&(all_df['School']=='Clemson')]
clem_mike_w_onerow=stats1row(['Mike Williams'],clem_mike_w,reccolumnsrename)
clem_mike_w_onerow[0]

#Get a dataframe consisting of only Mike Williams from Syracuse Info
syracuse_mike_w = all_df[(all_df['Player']=='Mike Williams')&(all_df['School']=='Syracuse')]
syracuse_mike_w_onerow=stats1row(['Mike Williams'],syracuse_mike_w,reccolumnsrename)
syracuse_mike_w_onerow[0]

#create dataframe of players collected with rushing stats first
rushing_first= pd.concat(p_no_recs_dfs)
# get unique list of player names to use in function
rush_players = rushing_first['Player'].unique()
#run dataframe through function to get 1 row per player
rush_df = stats1row(rush_players,rushing_first,rushcolumnsrename)
#Put togther list of dfs into 1
rush_to_rec_form = pd.concat(rush_df)
#Reorder the columns to join with the rec_guys dataframe
rush_final = rush_to_rec_form[['careergames', 'careerRec', 'careerRecYds', 'careerYPC', 'careerRecTds',
       'careerRushAtt', 'careerRushYds', 'careerYPA', 'careerRushTds',
       'careerPlays', 'careerTotalYds', 'careerYPT', 'careerTotalTds',
       'Player', 'Year', 'School', 'Conf', 'Class', 'Pos', 'dr_1games',
       'dr_1Rec', 'dr_1RecYds', 'dr_1YPC', 'dr_1RecTds', 'dr_1RushAtt',
       'dr_1RushYds', 'dr_1YPA', 'dr_1RushTds', 'dr_1Plays', 'dr_1TotalYds',
       'dr_1YPT', 'dr_1TotalTds', 'dr_2games', 'dr_2Rec', 'dr_2RecYds',
       'dr_2YPC', 'dr_2RecTds', 'dr_2RushAtt', 'dr_2RushYds', 'dr_2YPA',
       'dr_2RushTds', 'dr_2Plays', 'dr_2TotalYds', 'dr_2YPT', 'dr_2TotalTds',
       'dr_3games', 'dr_3Rec', 'dr_3RecYds', 'dr_3YPC', 'dr_3RecTds',
       'dr_3RushAtt', 'dr_3RushYds', 'dr_3YPA', 'dr_3RushTds', 'dr_3Plays',
       'dr_3TotalYds', 'dr_3YPT', 'dr_3TotalTds']]

#create df of all players that attended multiple schools
mult_schools = pd.concat(redo_dfs)
#get rid of subtotal rows by school
mult_schools['Year'].fillna('No Year',inplace=True)
mult_schools_final = mult_schools[mult_schools['Year']!= 'No Year']
#get unique list of players
mult_school_players = mult_schools_final['Player'].unique()
#convert to a single row
mult_schls = stats1row(mult_school_players,mult_schools_final,reccolumnsrename)
#concatinate list into 1 dataframe
mult_schools_one_row = pd.concat(mult_schls)
#save csv of mult_schools_data
mult_schools_one_row.to_csv("mult_schools.csv")

#Put together all dfs of players that played for a single team
all_college = pd.concat([minus_mikew,clem_mike_w_onerow[0],syracuse_mike_w_onerow[0],rush_to_rec_form,mult_schools_one_row])

#save to csv file
all_college.to_csv("allcollegestat.csv")

           players
statyears         
2                2
3               30
4              107
5              142
6               11
9                1
                   players  statyears
105  Cordarrelle Patterson          2
166            Kaelin Clay          2
                   players  statyears
105  Cordarrelle Patterson          2
166            Kaelin Clay          2
--------------------
--------------------
--------------------
--------------------
--------------------

C:\Users\walsh\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\walsh\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,



--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
------------

In [226]:
mult_schools

,Year,School,Conf,Class,Pos,G,Rec,Yds,Avg,TD,Att,Yds.1,Avg.1,TD.1,Plays,Yds.2,Avg.2,TD.2,Player
0,2006,Arkansas,SEC,FR,WR,13.0,19.0,235.0,12.4,2.0,1.0,5.0,5.0,0.0,20,240,12.0,2,Damian Williams
1,2008,USC,Pac-10,NaN,WR,13.0,58.0,869.0,15.0,9.0,4.0,43.0,10.8,0.0,62,912,14.7,9,Damian Williams
2,2009,USC,Pac-10,NaN,WR,12.0,70.0,1010.0,14.4,6.0,2.0,16.0,8.0,0.0,72,1026,14.3,6,Damian Williams
3,Career,Overall,NaN,NaN,NaN,NaN,147.0,2114.0,14.4,17.0,7.0,64.0,9.1,0.0,154,2178,14.1,17,Damian Williams
4,No Year,Arkansas,NaN,NaN,NaN,NaN,19.0,235.0,12.4,2.0,1.0,5.0,5.0,0.0,20,240,12.0,2,Damian Williams
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,2018,Florida,SEC,JR,WR,11.0,35.0,503.0,14.4,6.0,0.0,0.0,NaN,0.0,35,503,14.4,6,Van Jefferson
4,2019,Florida,SEC,SR,WR,12.0,49.0,657.0,13.4,6.0,1.0,7.0,7.0,0.0,50,664,13.3,6,Van Jefferson
5,Career,Overall,NaN,NaN,NaN,NaN,175.0,2159.0,12.3,16.0,6.0,47.0,7.8,0.0,181,2206,12.2,16,Van Jefferson
6,No Year,Ole Miss,NaN,NaN,NaN,NaN,91.0,999.0,11.0,4.0,5.0,40.0,8.0,0.0,96,1039,10.8,4,Van Jefferson
